In [1]:
import os
import h5py
import argparse
import numpy as np

from hdf5_utils import *

cars_hdf5_name = '../models/cars/2_watertight/sdf.h5'

h5 = read_hdf5(cars_hdf5_name)
print h5.shape

S = h5.reshape((h5.shape[0], -1))
# print S.shape

S_train = S[:, :]
# S_test = S[-5:, :]

S_mean = np.mean(S_train, axis=0)
S_white_train = S_train - S_mean
# S_white_test = S_test - S_mean
# print S_white_train.shape, S_white_test.shape, S_mean.shape
# t = S.shape[0] # number of samples, 87
# n = S.shape[1] # dim of feature, 32*32*32

from sklearn import decomposition
pca = decomposition.PCA(n_components=10)
pca.fit(S_white_train)
V = pca.components_.T # basis (mapping from high to low dim feature space), 32786*10
print V.shape

beta = np.dot(V.T, S_white_train.T) # embedding for 87 models, 10*87
print beta.shape

/home/rz1/.local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(79, 32, 32, 32)
(32768, 10)
(10, 79)


In [2]:
# Demo for compressing and recovering the 1st model.
# Assuming beta_input is the input PCA code with dimension [1, 10]
beta_input = beta[:, 0:1]
S_gnd = S_train[0:1, :]

S_recover = np.dot(V, beta_input).T+S_mean.reshape((1, -1))
print S_recover.shape

# The recovered shape and its ground truth shape will be written to .off files in original scale 
# as the dataset ('{model_id}_ori_scale.off')with Marching Cubes algorithm.
# The recovered model is at: ../models/cars/2_watertight/watertight/baojun-310-2017_ori_scale.off
# The reconstructed model is at: ../models/cars/2_watertight/recovered/mc_test_recon/baojun-310-2017_ori_scale.off

hdf5_recovered_path = '../models/cars/2_watertight/recovered/recovered_test_recon.h5'
write_hdf5(hdf5_recovered_path, S_recover.reshape((-1, 32, 32, 32)))
mc_recovered_path = hdf5_recovered_path.replace('recovered_test_recon.h5', 'mc_test_recon')
os.system('python marching_cubes.py %s %s'%(hdf5_recovered_path, mc_recovered_path))

hdf5_recovered_path = '../models/cars/2_watertight/recovered/recovered_test_gnd.h5'
write_hdf5(hdf5_recovered_path, S_gnd.reshape((-1, 32, 32, 32)))
mc_recovered_path = hdf5_recovered_path.replace('recovered_test_gnd.h5', 'mc_test_gnd')
os.system('python marching_cubes.py %s %s'%(hdf5_recovered_path, mc_recovered_path))

(1, 32768)


0